In [1]:
from data_extraction import db_handler as dbh
import pandas as pd

In [2]:
dbh.build_db_tables()

Fetching Events table...
Fetching Fixtures table...
Fetching player static info...
Fetching player detailed info...


100%|██████████| 591/591 [00:10<00:00, 55.94it/s]


In [36]:
db = dbh.DBHandler()
query = """
SELECT t.short_name as team, f.event_number, f.is_home_team, f.fixture_difficulty, opp.short_name as opponent
FROM NEXT_FIVE_FIXTURES_BY_TEAM_VIEW f
JOIN TEAMS_STATIC opp
ON f.opponent_team = opp.id
JOIN TEAMS_STATIC t
on f.team_id = t.id
"""
df = pd.read_sql_query(query, db.conn)
df.head()

,team,event_number,is_home_team,fixture_difficulty,opponent
0,ARS,1,1,2,FUL
1,ARS,2,1,2,AVL
2,ARS,3,0,4,MUN
3,ARS,4,1,2,EVE
4,ARS,5,0,2,BRE


In [44]:
fixtures_flat = pd.pivot(df, index='team', columns='event_number')
fixtures_flat.columns = fixtures_flat.columns.map(lambda x: f"{x[0]}_{x[1]}")
fixtures_flat

,is_home_team_1,is_home_team_2,is_home_team_3,is_home_team_4,is_home_team_5,fixture_difficulty_1,fixture_difficulty_2,fixture_difficulty_3,fixture_difficulty_4,fixture_difficulty_5,opponent_1,opponent_2,opponent_3,opponent_4,opponent_5
team,,,,,,,,,,,,,,,
ARS,1,1,0,1,0,2,2,4,2,2,FUL,AVL,MUN,EVE,BRE
AVL,1,0,1,0,1,3,4,5,4,2,WHU,ARS,MCI,LEI,SOU
BHA,1,0,1,0,1,2,2,3,2,2,LEE,FUL,LEI,BOU,CRY
BOU,0,1,0,1,0,5,2,2,2,3,LIV,WOL,NFO,BHA,NEW
BRE,1,0,1,0,1,2,2,2,2,3,EVE,CRY,LEE,SOU,ARS
CHE,1,0,1,0,1,3,2,3,2,5,LEI,SOU,WHU,FUL,LIV
CRY,0,1,0,1,0,5,2,3,3,2,MCI,BRE,NEW,MUN,BHA
EVE,0,0,1,0,1,2,2,5,4,3,BRE,LEE,LIV,ARS,WHU
FUL,0,1,0,1,0,4,2,4,4,2,ARS,BHA,TOT,CHE,NFO


In [46]:
pd.read_sql_query("SELECT * FROM PLAYER_TABULAR_VIEW", db.conn).merge(fixtures_flat, how="inner", left_on="team_name", right_on="team")

,web_name,position,team_name,total_points,now_cost,selected_by_percent,event_points,goals_scored,assists,goal_contribution,...,fixture_difficulty_1,fixture_difficulty_2,fixture_difficulty_3,fixture_difficulty_4,fixture_difficulty_5,opponent_1,opponent_2,opponent_3,opponent_4,opponent_5
0,Cédric,DEF,ARS,0,43,0.2,0,0,0,0,...,2,2,4,2,2,FUL,AVL,MUN,EVE,BRE
1,Xhaka,MID,ARS,20,50,2.5,6,1,2,3,...,2,2,4,2,2,FUL,AVL,MUN,EVE,BRE
2,Elneny,MID,ARS,0,44,1.4,0,0,0,0,...,2,2,4,2,2,FUL,AVL,MUN,EVE,BRE
3,Holding,DEF,ARS,0,43,0.1,0,0,0,0,...,2,2,4,2,2,FUL,AVL,MUN,EVE,BRE
4,Partey,MID,ARS,8,50,0.6,3,0,0,0,...,2,2,4,2,2,FUL,AVL,MUN,EVE,BRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,Hodge,MID,WOL,0,45,0.0,0,0,0,0,...,3,2,2,5,5,NEW,BOU,SOU,LIV,MCI
587,Ronan,MID,WOL,0,45,0.1,0,0,0,0,...,3,2,2,5,5,NEW,BOU,SOU,LIV,MCI
588,Campbell,MID,WOL,1,45,0.3,0,0,0,0,...,3,2,2,5,5,NEW,BOU,SOU,LIV,MCI
589,Guedes,MID,WOL,3,60,0.2,2,0,0,0,...,3,2,2,5,5,NEW,BOU,SOU,LIV,MCI
